# OpenVINO™ Runtime API Tutorial

This notebook explains the basics of the OpenVINO Runtime API.

The notebook is divided into sections with headers. The next cell contains global requirements for installation and imports. Each section is standalone and does not depend on any previous sections. All models used in this tutorial are provided as examples. These model files can be replaced with your own models. The exact outputs will be different, but the process is the same.


#### Table of contents:

- [Loading OpenVINO Runtime and Showing Info](#Loading-OpenVINO-Runtime-and-Showing-Info)
- [Loading a Model](#Loading-a-Model)
    - [OpenVINO IR Model](#OpenVINO-IR-Model)
    - [ONNX Model](#ONNX-Model)
    - [PaddlePaddle Model](#PaddlePaddle-Model)
    - [TensorFlow Model](#TensorFlow-Model)
    - [TensorFlow Lite Model](#TensorFlow-Lite-Model)
    - [PyTorch Model](#PyTorch-Model)
- [Getting Information about a Model](#Getting-Information-about-a-Model)
    - [Model Inputs](#Model-Inputs)
    - [Model Outputs](#Model-Outputs)
- [Doing Inference on a Model](#Doing-Inference-on-a-Model)
- [Reshaping and Resizing](#Reshaping-and-Resizing)
    - [Change Image Size](#Change-Image-Size)
    - [Change Batch Size](#Change-Batch-Size)
- [Caching a Model](#Caching-a-Model)


### Installation Instructions

This is a self-contained example that relies solely on its own code.

We recommend  running the notebook in a virtual environment. You only need a Jupyter server to start.
For details, please refer to [Installation Guide](https://github.com/openvinotoolkit/openvino_notebooks/blob/latest/README.md#-installation-guide).

<img referrerpolicy="no-referrer-when-downgrade" src="https://static.scarf.sh/a.png?x-pxid=5b5a4db0-7875-4bfb-bdbd-01698b5b1a77&file=notebooks/openvino-api/openvino-api.ipynb" />


In [ ]:
import platform

# Required imports. Please execute this cell first.
%pip install -q "openvino>=2023.1.0"
%pip install -q requests tqdm ipywidgets

if platform.system() == "Darwin":
    %pip install -q "numpy<2.0.0"

# Fetch `notebook_utils` module
import requests
from pathlib import Path

if not Path("notebook_utils.py").exists():
    r = requests.get(
        url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/notebook_utils.py",
    )

    open("notebook_utils.py", "w").write(r.text)

from notebook_utils import download_file, device_widget

# Read more about telemetry collection at https://github.com/openvinotoolkit/openvino_notebooks?tab=readme-ov-file#-telemetry
from notebook_utils import collect_telemetry

collect_telemetry("openvino-api.ipynb")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Loading OpenVINO Runtime and Showing Info
[back to top ⬆️](#Table-of-contents:)

Initialize OpenVINO Runtime with `ov.Core()`

In [ ]:
import openvino as ov

core = ov.Core()

OpenVINO Runtime can load a network on a device. A device in this context means a CPU, an Intel GPU, a Neural Compute Stick 2, etc. The `available_devices` property shows the available devices in your system. The "FULL_DEVICE_NAME" option to `core.get_property()` shows the name of the device.

In [ ]:
import openvino.properties as props


devices = core.available_devices

for device in devices:
    device_name = core.get_property(device, props.device.full_name)
    print(f"{device}: {device_name}")

CPU: Intel(R) Core(TM) i9-10980XE CPU @ 3.00GHz


### Select device for inference

You can specify which device from available devices will be used for inference using this widget

In [ ]:
device = device_widget()

device

Dropdown(description='Device:', index=1, options=('CPU', 'AUTO'), value='AUTO')

## Loading a Model
[back to top ⬆️](#Table-of-contents:)

After initializing OpenVINO Runtime, first read the model file with `read_model()`, then compile it to the specified device with the `compile_model()` method.

[OpenVINO™ supports several model formats](https://docs.openvino.ai/2024/openvino-workflow/model-preparation/convert-model-to-ir.html) and enables developers to convert them to its own OpenVINO IR format using a tool dedicated to this task.

### OpenVINO IR Model
[back to top ⬆️](#Table-of-contents:)

An OpenVINO IR (Intermediate Representation) model consists of an `.xml` file, containing information about network topology, and a `.bin` file, containing the weights and biases binary data. Models in OpenVINO IR format are obtained by using model conversion API. The `read_model()` function expects the `.bin` weights file to have the same filename and be located in the same directory as the `.xml` file: `model_weights_file == Path(model_xml).with_suffix(".bin")`. If this is the case, specifying the weights file is optional. If the weights file has a different filename, it can be specified using the `weights` parameter in `read_model()`.

The OpenVINO [Model Conversion API](https://docs.openvino.ai/2024/openvino-workflow/model-preparation.html) tool is used to convert models to OpenVINO IR format. Model conversion API reads the original model and creates an OpenVINO IR model (`.xml` and `.bin` files) so inference can be performed without delays due to format conversion. Optionally, model conversion API can adjust the model to be more suitable for inference, for example, by alternating input shapes, embedding preprocessing and cutting training parts off.
For information on how to convert your existing TensorFlow, PyTorch model to OpenVINO IR format with model conversion API, refer to the [tensorflow-to-openvino](../tensorflow-classification-to-openvino/tensorflow-classification-to-openvino.ipynb) and [pytorch-to-openvino](../pytorch-to-openvino/pytorch-to-openvino.ipynb) notebooks.

In [ ]:
ir_model_url = "https://storage.openvinotoolkit.org/repositories/openvino_notebooks/models/002-example-models/"
ir_model_name_xml = "classification.xml"
ir_model_name_bin = "classification.bin"

if not Path("model/" + ir_model_name_xml).exists():
    download_file(ir_model_url + ir_model_name_xml, filename=ir_model_name_xml, directory="model")
    download_file(ir_model_url + ir_model_name_bin, filename=ir_model_name_bin, directory="model")

classification.xml:   0%|          | 0.00/179k [00:00<?, ?B/s]

classification.bin:   0%|          | 0.00/4.84M [00:00<?, ?B/s]

In [ ]:
import openvino as ov

core = ov.Core()
classification_model_xml = "model/classification.xml"

model = core.read_model(model=classification_model_xml)
compiled_model = core.compile_model(model=model, device_name=device.value)

### ONNX Model
[back to top ⬆️](#Table-of-contents:)

[ONNX](https://onnx.ai/) is an open format built to represent machine learning models. ONNX defines a common set of operators - the building blocks of machine learning and deep learning models - and a common file format to enable AI developers to use models with a variety of frameworks, tools, runtimes, and compilers. OpenVINO supports reading models in ONNX format directly,that means they can be used with OpenVINO Runtime without any prior conversion.

Reading and loading an ONNX model, which is a single `.onnx` file, works the same way as with an OpenVINO IR model. The `model` argument points to the filename of an ONNX model.

In [ ]:
onnx_model_url = "https://storage.openvinotoolkit.org/repositories/openvino_notebooks/models/002-example-models/segmentation.onnx"
onnx_model_name = "segmentation.onnx"

if not Path("model/" + onnx_model_name).exists():
    download_file(onnx_model_url, filename=onnx_model_name, directory="model")

segmentation.onnx:   0%|          | 0.00/4.41M [00:00<?, ?B/s]

In [ ]:
import openvino as ov

core = ov.Core()
onnx_model_path = "model/segmentation.onnx"

model_onnx = core.read_model(model=onnx_model_path)
compiled_model_onnx = core.compile_model(model=model_onnx, device_name=device.value)

The ONNX model can be exported to OpenVINO IR with `save_model()`:

In [ ]:
ov.save_model(model_onnx, output_model="model/exported_onnx_model.xml")

### PaddlePaddle Model
[back to top ⬆️](#Table-of-contents:)

[PaddlePaddle](https://www.paddlepaddle.org.cn/documentation/docs/en/guides/index_en.html) models saved for inference can also be passed to OpenVINO Runtime without any conversion step. Pass the filename with extension to `read_model` and exported an OpenVINO IR with `save_model`


In [ ]:
paddle_model_url = "https://storage.openvinotoolkit.org/repositories/openvino_notebooks/models/002-example-models/"
paddle_model_name = "inference.pdmodel"
paddle_params_name = "inference.pdiparams"

if not Path("model/" + paddle_model_name).exists() or not Path("model/" + paddle_params_name).exists():
    download_file(paddle_model_url + paddle_model_name, filename=paddle_model_name, directory="model")
    download_file(
        paddle_model_url + paddle_params_name,
        filename=paddle_params_name,
        directory="model",
    )

inference.pdmodel:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

inference.pdiparams:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

In [ ]:
import openvino as ov

core = ov.Core()
paddle_model_path = "model/inference.pdmodel"

model_paddle = core.read_model(model=paddle_model_path)
compiled_model_paddle = core.compile_model(model=model_paddle, device_name=device.value)

In [ ]:
ov.save_model(model_paddle, output_model="model/exported_paddle_model.xml")

### TensorFlow Model
[back to top ⬆️](#Table-of-contents:)

TensorFlow models saved in frozen graph format can also be passed to `read_model`.


In [ ]:
pb_model_url = "https://storage.openvinotoolkit.org/repositories/openvino_notebooks/models/002-example-models/classification.pb"
pb_model_name = "classification.pb"

if not Path("model/" + pb_model_name).exists():
    download_file(pb_model_url, filename=pb_model_name, directory="model")

classification.pb:   0%|          | 0.00/9.88M [00:00<?, ?B/s]

In [ ]:
import openvino as ov

core = ov.Core()
tf_model_path = "model/classification.pb"

model_tf = core.read_model(model=tf_model_path)
compiled_model_tf = core.compile_model(model=model_tf, device_name=device.value)

In [ ]:
ov.save_model(model_tf, output_model="model/exported_tf_model.xml")

### TensorFlow Lite Model
[back to top ⬆️](#Table-of-contents:)

[TFLite](https://www.tensorflow.org/lite) models saved for inference can also be passed to OpenVINO Runtime. Pass the filename with extension `.tflite` to `read_model` and exported an OpenVINO IR with `save_model`.

This tutorial uses the image classification model [inception_v4_quant](https://tfhub.dev/tensorflow/lite-model/inception_v4_quant/1/default/1). It is pre-trained model optimized to work with TensorFlow Lite.

In [ ]:
%pip install -q kagglehub

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pathlib import Path
import kagglehub

tflite_model_dir = kagglehub.model_download("tensorflow/inception/tfLite/v4-quant")
tflite_model_path = Path(tflite_model_dir) / "1.tflite"


  0%|                                               | 0.00/40.9M [00:00<?, ?B/s]
  2%|▉                                     | 1.00M/40.9M [00:00<00:34, 1.23MB/s]
  5%|█▊                                    | 2.00M/40.9M [00:01<00:18, 2.22MB/s]
  7%|██▊                                   | 3.00M/40.9M [00:01<00:12, 3.28MB/s]
 10%|███▋                                  | 4.00M/40.9M [00:01<00:09, 4.20MB/s]
 12%|████▋                                 | 5.00M/40.9M [00:01<00:07, 5.10MB/s]
 15%|█████▌                                | 6.00M/40.9M [00:01<00:06, 5.95MB/s]
 17%|██████▌                               | 7.00M/40.9M [00:01<00:05, 6.78MB/s]
 22%|████████▎                             | 9.00M/40.9M [00:01<00:03, 8.47MB/s]
 27%|██████████▏                           | 11.0M/40.9M [00:02<00:03, 9.81MB/s]
 32%|████████████                          | 13.0M/40.9M [00:02<00:02, 11.3MB/s]
 37%|█████████████▉                        | 15.0M/40.9M [00:02<00:02, 12.6MB/s]
 42%|███████████████▊      

In [ ]:
import openvino as ov

core = ov.Core()

model_tflite = core.read_model(tflite_model_path)
compiled_model_tflite = core.compile_model(model=model_tflite, device_name=device.value)

In [ ]:
ov.save_model(model_tflite, output_model="model/exported_tflite_model.xml")

### PyTorch Model
[back to top ⬆️](#Table-of-contents:)

[PyTorch](https://pytorch.org/) models can not be directly passed to `core.read_model`. `ov.Model` for model objects from this framework can be obtained using `ov.convert_model` API. You can find more details in [pytorch-to-openvino](../pytorch-to-openvino) notebook. In this tutorial we will use [resnet18](https://pytorch.org/vision/main/models/generated/torchvision.models.resnet18.html) model form torchvision library. After conversion model using `ov.convert_model`, it can be compiled on device using `core.compile_model` or saved on disk for the next usage using `ov.save_model`


In [ ]:
%pip install -q "torch>=2.1" torchvision --extra-index-url https://download.pytorch.org/whl/cpu

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import openvino as ov
import torch
from torchvision.models import resnet18, ResNet18_Weights

core = ov.Core()

pt_model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
example_input = torch.zeros((1, 3, 224, 224))
ov_model_pytorch = ov.convert_model(pt_model, example_input=example_input)

compiled_model_pytorch = core.compile_model(ov_model_pytorch, device_name=device.value)

ov.save_model(ov_model_pytorch, "model/exported_pytorch_model.xml")

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/labuser/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████████████████████████████████| 44.7M/44.7M [00:02<00:00, 18.3MB/s]


## Getting Information about a Model
[back to top ⬆️](#Table-of-contents:)

The OpenVINO Model instance stores information about the model. Information about the inputs and outputs of the model are in `model.inputs` and `model.outputs`. These are also properties of the `CompiledModel` instance. While using `model.inputs` and `model.outputs` in the cells below, you can also use `compiled_model.inputs` and `compiled_model.outputs`.

In [ ]:
ir_model_url = "https://storage.openvinotoolkit.org/repositories/openvino_notebooks/models/002-example-models/"
ir_model_name_xml = "classification.xml"
ir_model_name_bin = "classification.bin"

if not Path("model/" + ir_model_name_xml).exists():
    download_file(ir_model_url + ir_model_name_xml, filename=ir_model_name_xml, directory="model")
    download_file(ir_model_url + ir_model_name_bin, filename=ir_model_name_bin, directory="model")

### Model Inputs
[back to top ⬆️](#Table-of-contents:)

Information about all input layers is stored in the `inputs` dictionary.

In [ ]:
import openvino as ov

core = ov.Core()
classification_model_xml = "model/classification.xml"
model = core.read_model(model=classification_model_xml)
model.inputs

[<Output: names[input, input:0] shape[1,3,224,224] type: f32>]

The cell above shows that the loaded model expects one input with the name _input_. If you loaded a different model, you may see a different input layer name, and you may see more inputs. You may also obtain info about each input layer using `model.input(index)`, where index is a numeric index of the input layers in the model. If a model has only one input, index can be omitted.

In [ ]:
input_layer = model.input(0)

It is often useful to have a reference to the name of the first input layer. For a model with one input, `model.input(0).any_name` gets this name.

In [ ]:
input_layer.any_name

'input'

The next cell prints the input layout, precision and shape.

In [ ]:
print(f"input precision: {input_layer.element_type}")
print(f"input shape: {input_layer.shape}")

input precision: <Type: 'float32'>
input shape: [1,3,224,224]


This cell shows that the model expects inputs with a shape of [1,3,224,224], and that this is in the `NCHW` layout. This means that the model expects input data with the batch size of 1 (`N`), 3 channels (`C`) , and images with a height (`H`) and width (`W`) equal to 224. The input data is expected to be of `FP32` (floating point) precision.

### Model Outputs
[back to top ⬆️](#Table-of-contents:)


In [ ]:
import openvino as ov

core = ov.Core()
classification_model_xml = "model/classification.xml"
model = core.read_model(model=classification_model_xml)
model.outputs

[<Output: names[MobilenetV3/Predictions/Softmax] shape[1,1001] type: f32>]

Model output info is stored in `model.outputs`. The cell above shows that the model returns one output, with the `MobilenetV3/Predictions/Softmax` name. Loading a different model will result in different output layer name, and more outputs might be returned. Similar to input, you may also obtain information about each output separately using `model.output(index)`

Since this model has one output, follow the same method as for the input layer to get its name.

In [ ]:
output_layer = model.output(0)
output_layer.any_name

'MobilenetV3/Predictions/Softmax'

Getting the output precision and shape is similar to getting the input precision and shape.

In [ ]:
print(f"output precision: {output_layer.element_type}")
print(f"output shape: {output_layer.shape}")

output precision: <Type: 'float32'>
output shape: [1,1001]


This cell shows that the model returns outputs with a shape of [1, 1001], where 1 is the batch size (`N`) and 1001 is the number of classes (`C`). The output is returned as 32-bit floating point.

## Doing Inference on a Model
[back to top ⬆️](#Table-of-contents:)

> **NOTE** this notebook demonstrates only the basic synchronous inference API. For an async inference example, please refer to [Async API notebook](../async-api/async-api.ipynb)

The diagram below shows a typical inference pipeline with OpenVINO

![image.png](https://github.com/openvinotoolkit/openvino_notebooks/assets/29454499/a91bc582-165b-41a2-ab08-12c812059936)

Creating OpenVINO Core and model compilation is covered in the previous steps. The next step is preparing inputs. You can provide inputs in one of the supported format: dictionary with name of inputs as keys and `np.arrays` that represent input tensors as values, list or tuple of `np.arrays` represented input tensors (their order should match with model inputs order). If a model has a single input, wrapping to a dictionary or list can be omitted.   To do inference on a model, pass prepared inputs into compiled model object obtained using `core.compile_model`. The inference result represented as dictionary, where keys are model outputs and `np.arrays` represented their produced data as values.  

In [ ]:
# Install opencv package for image handling
%pip install -q opencv-python

Note: you may need to restart the kernel to use updated packages.


**Load the network**

In [ ]:
ir_model_url = "https://storage.openvinotoolkit.org/repositories/openvino_notebooks/models/002-example-models/"
ir_model_name_xml = "classification.xml"
ir_model_name_bin = "classification.bin"

if not Path("model/" + ir_model_name_xml).exists():
    download_file(ir_model_url + ir_model_name_xml, filename=ir_model_name_xml, directory="model")
    download_file(ir_model_url + ir_model_name_bin, filename=ir_model_name_bin, directory="model")

In [ ]:
import openvino as ov

core = ov.Core()
classification_model_xml = "model/classification.xml"
model = core.read_model(model=classification_model_xml)
compiled_model = core.compile_model(model=model, device_name=device.value)
input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)

**Load an image and convert to the input shape**

To propagate an image through the network, it needs to be loaded into an array, resized to the shape that the network expects, and converted to the input layout of the network.

In [ ]:
import cv2

image_filename = Path("data/coco_hollywood.jpg")
if not image_filename.exists():
    image_filename = download_file(
        "https://storage.openvinotoolkit.org/repositories/openvino_notebooks/data/data/image/coco_hollywood.jpg",
        directory="data",
    )
image = cv2.imread(str(image_filename))
image.shape

coco_hollywood.jpg:   0%|          | 0.00/485k [00:00<?, ?B/s]

(663, 994, 3)

The image has a shape of (663,994,3). It is 663 pixels in height, 994 pixels in width, and has 3 color channels. A reference to the height and width expected by the network is obtained and the image is resized to these dimensions.

In [ ]:
# N,C,H,W = batch size, number of channels, height, width.
N, C, H, W = input_layer.shape
# OpenCV resize expects the destination size as (width, height).
resized_image = cv2.resize(src=image, dsize=(W, H))
resized_image.shape

(224, 224, 3)

Now, the image has the width and height that the network expects. This is still in `HWC` format and must be changed to `NCHW` format. First, call the `np.transpose()` method to change to `CHW` and then add the `N` dimension (where `N`= 1) by calling the `np.expand_dims()` method. Next, convert the data to `FP32` with `np.astype()` method.

In [ ]:
import numpy as np

input_data = np.expand_dims(np.transpose(resized_image, (2, 0, 1)), 0).astype(np.float32)
input_data.shape

(1, 3, 224, 224)

**Do inference**

Now that the input data is in the right shape, run inference. The `CompiledModel` inference result is a dictionary where keys are the Output class instances (the same keys in `compiled_model.outputs` that can also be obtained with `compiled_model.output(index)`) and values - predicted result in `np.array` format.

In [ ]:
# for single input models only
result = compiled_model(input_data)[output_layer]

# for multiple inputs in a list
result = compiled_model([input_data])[output_layer]

# or using a dictionary, where the key is input tensor name or index
result = compiled_model({input_layer.any_name: input_data})[output_layer]

You can also create `InferRequest` and run `infer` method on request.

In [ ]:
request = compiled_model.create_infer_request()
request.infer(inputs={input_layer.any_name: input_data})
result = request.get_output_tensor(output_layer.index).data

The `.infer()` function sets output tensor, that can be reached, using `get_output_tensor()`. Since this network returns one output, and the reference to the output layer is in the `output_layer.index` parameter, you can get the data with `request.get_output_tensor(output_layer.index)`. To get a numpy array from the output, use the `.data` parameter.

In [ ]:
result.shape

(1, 1001)

The output shape is (1,1001), which is the expected output shape. This shape indicates that the network returns probabilities for 1001 classes. To learn more about this notion, refer to the [hello world notebook](../hello-world/hello-world.ipynb).

## Reshaping and Resizing
[back to top ⬆️](#Table-of-contents:)

### Change Image Size
[back to top ⬆️](#Table-of-contents:)


Instead of reshaping the image to fit the model, it is also possible to reshape the model to fit the image. Be aware that not all models support reshaping, and models that do, may not support all input shapes. The model accuracy may also suffer if you reshape the model input shape.

First check the input shape of the model, then reshape it to the new input shape.

In [ ]:
ir_model_url = "https://storage.openvinotoolkit.org/repositories/openvino_notebooks/models/002-example-models/"
ir_model_name_xml = "segmentation.xml"
ir_model_name_bin = "segmentation.bin"

if not Path("model/" + ir_model_name_xml).exists():
    download_file(ir_model_url + ir_model_name_xml, filename=ir_model_name_xml, directory="model")
    download_file(ir_model_url + ir_model_name_bin, filename=ir_model_name_bin, directory="model")

segmentation.xml:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

segmentation.bin:   0%|          | 0.00/1.09M [00:00<?, ?B/s]

In [ ]:
import openvino as ov

core = ov.Core()
segmentation_model_xml = "model/segmentation.xml"
segmentation_model = core.read_model(model=segmentation_model_xml)
segmentation_input_layer = segmentation_model.input(0)
segmentation_output_layer = segmentation_model.output(0)

print("~~~~ ORIGINAL MODEL ~~~~")
print(f"input shape: {segmentation_input_layer.shape}")
print(f"output shape: {segmentation_output_layer.shape}")

new_shape = ov.PartialShape([1, 3, 544, 544])
segmentation_model.reshape({segmentation_input_layer.any_name: new_shape})
segmentation_compiled_model = core.compile_model(model=segmentation_model, device_name=device.value)
# help(segmentation_compiled_model)
print("~~~~ RESHAPED MODEL ~~~~")
print(f"model input shape: {segmentation_input_layer.shape}")
print(f"compiled_model input shape: " f"{segmentation_compiled_model.input(index=0).shape}")
print(f"compiled_model output shape: {segmentation_output_layer.shape}")

~~~~ ORIGINAL MODEL ~~~~
input shape: [1,3,512,512]
output shape: [1,1,512,512]
~~~~ RESHAPED MODEL ~~~~
model input shape: [1,3,544,544]
compiled_model input shape: [1,3,544,544]
compiled_model output shape: [1,1,544,544]


The input shape for the segmentation network is [1,3,512,512], with the `NCHW` layout: the network expects 3-channel images with a width and height of 512 and a batch size of 1. Reshape the network with the `.reshape()` method of `IENetwork` to make it accept input images with a width and height of 544. This segmentation network always returns arrays with the input width and height of equal value. Therefore, setting the input dimensions to 544x544 also modifies the output dimensions. After reshaping, compile the network once again.

### Change Batch Size
[back to top ⬆️](#Table-of-contents:)


Use the `.reshape()` method to set the batch size, by increasing the first element of `new_shape`. For example, to set a batch size of two, set `new_shape = (2,3,544,544)` in the cell above.

In [ ]:
import openvino as ov

segmentation_model_xml = "model/segmentation.xml"
segmentation_model = core.read_model(model=segmentation_model_xml)
segmentation_input_layer = segmentation_model.input(0)
segmentation_output_layer = segmentation_model.output(0)
new_shape = ov.PartialShape([2, 3, 544, 544])
segmentation_model.reshape({segmentation_input_layer.any_name: new_shape})
segmentation_compiled_model = core.compile_model(model=segmentation_model, device_name=device.value)

print(f"input shape: {segmentation_input_layer.shape}")
print(f"output shape: {segmentation_output_layer.shape}")

input shape: [2,3,544,544]
output shape: [2,1,544,544]


The output shows that by setting the batch size to 2, the first element (`N`) of the input and output shape has a value of 2. Propagate the input image through the network to see the result:

In [ ]:
import numpy as np
import openvino as ov

core = ov.Core()
segmentation_model_xml = "model/segmentation.xml"
segmentation_model = core.read_model(model=segmentation_model_xml)
segmentation_input_layer = segmentation_model.input(0)
segmentation_output_layer = segmentation_model.output(0)
new_shape = ov.PartialShape([2, 3, 544, 544])
segmentation_model.reshape({segmentation_input_layer.any_name: new_shape})
segmentation_compiled_model = core.compile_model(model=segmentation_model, device_name=device.value)
input_data = np.random.rand(2, 3, 544, 544)

output = segmentation_compiled_model([input_data])

print(f"input data shape: {input_data.shape}")
print(f"result data data shape: {segmentation_output_layer.shape}")

input data shape: (2, 3, 544, 544)
result data data shape: [2,1,544,544]


## Caching a Model
[back to top ⬆️](#Table-of-contents:)

For some devices, like GPU, loading a model can take some time. Model Caching solves this issue by caching the model in a cache directory. If `core.compile_model(model=net, device_name=device_name, config=config_dict)` is set, caching will be used. This option checks if a model exists in the cache. If so, it loads it from the cache. If not, it loads the model regularly, and stores it in the cache, so that the next time the model is loaded when this option is set, the model will be loaded from the cache.

In the cell below, we create a *model_cache* directory as a subdirectory of *model*, where the model will be cached for the specified device. The model will be loaded to the GPU. After running this cell once, the model will be cached, so subsequent runs of this cell will load the model from the cache.

*Note: Model Caching is also available on CPU devices*

In [ ]:
ir_model_url = "https://storage.openvinotoolkit.org/repositories/openvino_notebooks/models/002-example-models/"
ir_model_name_xml = "classification.xml"
ir_model_name_bin = "classification.bin"

if not Path("model/" + ir_model_name_xml).exists():
    download_file(ir_model_url + ir_model_name_xml, filename=ir_model_name_xml, directory="model")
    download_file(ir_model_url + ir_model_name_bin, filename=ir_model_name_bin, directory="model")

In [ ]:
import time
from pathlib import Path

import openvino as ov

core = ov.Core()

cache_path = Path("model/model_cache")
cache_path.mkdir(exist_ok=True)
# Enable caching for OpenVINO Runtime. To disable caching set enable_caching = False
enable_caching = True
config_dict = {"CACHE_DIR": str(cache_path)} if enable_caching else {}

classification_model_xml = "model/classification.xml"
model = core.read_model(model=classification_model_xml)

start_time = time.perf_counter()
compiled_model = core.compile_model(model=model, device_name=device.value, config=config_dict)
end_time = time.perf_counter()
print(f"Loading the network to the {device.value} device took {end_time-start_time:.2f} seconds.")

Loading the network to the AUTO device took 0.13 seconds.


After running the previous cell, we know the model exists in the cache directory. Then, we delete the compiled model and load it again. Now, we measure the time it takes now.

In [ ]:
del compiled_model
start_time = time.perf_counter()
compiled_model = core.compile_model(model=model, device_name=device.value, config=config_dict)
end_time = time.perf_counter()
print(f"Loading the network to the {device.value} device took {end_time-start_time:.2f} seconds.")

Loading the network to the AUTO device took 0.06 seconds.


In [52]:
pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 23.9 MB/s eta 0:00:00


In [53]:

pip install pyaudio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pyaudio)


In [55]:
pip install pipwin


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.8 MB/s eta 0:00:00
  Created wheel for pipwin: filename=pipwin-0.5.2-py2.py3-none-any.whl size=8770 sha256=b2bdb332db0b7081f258e50f41e71ed93c7453084c52d08dbdf238df0a8b631a
  Stored in directory: /root/.cache/pip/wheels/07/45/2e/51ee464b7c3407327439df34c2fc62f978a1e5084aaf23d201
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=11352654c7a0988a191c7c1523954f833e375ea82005532bbbd29959dc909eaf
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
  Created wheel for pyjsparser: filename=pyjsparser-2.7.1-py3-none-any.whl size=25982 sha256=57dfe0fe7e906bfb2db7586a8e7e0fbc75c53f0f60f37117ea01f529806283a1
  Stored in directory: /root/.cache/pip/wheels/a5/9a/30/1003e89ab4555b81840ca46d361bf184f1e6ad880cae3b62a9
Successfully b

In [57]:
pip install pyaudio

  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pyaudio)


In [58]:
pip install pipwin

In [59]:
pipwin install pyaudio

SyntaxError: invalid syntax (ipython-input-59-1881691980.py, line 1)

In [54]:
pip install pyaudio


  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pyaudio)


In [60]:
import speech_recognition as sr

# Initialize recognizer
recognizer = sr.Recognizer()

# Use the microphone as source
with sr.Microphone() as source:
    print("Speak something...")
    audio = recognizer.listen(source)

    try:
        # Convert speech to text
        text = recognizer.recognize_google(audio)
        print("You said:", text)
    except sr.UnknownValueError:
        print("Sorry, could not understand your speech.")
    except sr.RequestError as e:
        print(f"Could not request results; {e}")


AttributeError: Could not find PyAudio; check installation

In [61]:
!apt-get install -y portaudio19-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (242 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 126281 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.

In [62]:
!pip install pyaudio


  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=pyaudio-0.2.14-cp311-cp311-linux_x86_64.whl size=67423 sha256=00ce90af06ee0ea933e3aee6ce64efdcf8375ac9ff41dae12c10fa93998eb2da
  Stored in directory: /root/.cache/pip/wheels/80/b1/c1/67e4ef443de2665d86031d4760508094eab5de37d5d64d9c27
Successfully built pyaudio


In [67]:
pip install pipwin


In [64]:
pip install speechrecognition pyttsx3 pyaudio



SyntaxError: invalid syntax (ipython-input-64-2493101017.py, line 1)

In [68]:
!pip install speechrecognition pyttsx3 pyaudio



In [69]:
import speech_recognition as sr
import pyttsx3

# Initialize recognizer and text-to-speech engine
recognizer = sr.Recognizer()
tts_engine = pyttsx3.init()

# Function: Convert text to speech
def speak(text):
    tts_engine.say(text)
    tts_engine.runAndWait()

# Function: Listen to microphone and convert speech to text
def listen():
    with sr.Microphone() as source:
        print("🎤 Please speak something...")
        audio = recognizer.listen(source)

        try:
            text = recognizer.recognize_google(audio)
            print("📝 You said:", text)
            return text
        except sr.UnknownValueError:
            print("😕 Sorry, I could not understand the audio.")
            return ""
        except sr.RequestError as e:
            print("❌ Could not request results; check your internet connection.")
            return ""

# Main Execution
if __name__ == "__main__":
    spoken_text = listen()
    if spoken_text:
        print("🔊 Speaking it back to you...")
        speak(spoken_text)


RuntimeError: This means you probably do not have eSpeak or eSpeak-ng installed!

In [74]:
!apt-get install -y portaudio19-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
portaudio19-dev is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [75]:
tts_engine = pyttsx3.init()


RuntimeError: This means you probably do not have eSpeak or eSpeak-ng installed!

In [76]:
!apt-get install -y espeak


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  espeak-data libespeak1 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libsonic0
0 upgraded, 4 newly installed, 0 to remove and 35 not upgraded.
Need to get 1,316 kB of archives.
After this operation, 2,955 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsonic0 amd64 0.2.0-11build1 [10.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 espeak-data amd64 1.48.15+dfsg-3 [1,085 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libespeak1 amd64 1.48.15+dfsg-3 [156 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 espeak amd64 1.48.15+dfsg-3 [64.2 kB]
Fetched 1,316 kB in 1s (1,380 kB/s)
Selecting previously unselected package libsonic0:amd64.
(Reading database ... 126325 files and directories currently insta

In [77]:
import pyttsx3

tts_engine = pyttsx3.init()
tts_engine.say("Hello, how are you?")
tts_engine.runAndWait()


ValueError: SetVoiceByName failed with unknown return code -1 for voice: gmw/en

In [78]:
!pip install gTTS


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [80]:
from gtts import gTTS
from IPython.display import Audio

text = "Hello, how are you?, what's your name?"

# Convert to audio using Google Text-to-Speech
tts = gTTS(text=text, lang='en')
tts.save("speech.mp3")

# Play the audio in notebook
Audio("speech.mp3", autoplay=True)


In [81]:
import speech_recognition as sr

# Initialize recognizer
recognizer = sr.Recognizer()

# Use the microphone as source
with sr.Microphone() as source:
    print("🎤 Speak something...")
    audio = recognizer.listen(source)

    try:
        # Convert speech to text using Google Web Speech API
        text = recognizer.recognize_google(audio)
        print("📝 You said:", text)
    except sr.UnknownValueError:
        print("❌ Sorry, could not understand the audio.")
    except sr.RequestError:
        print("⚠️ Could not request results from Google Speech Recognition service.")


OSError: No Default Input Device Available

In [83]:
pip install SpeechRecognition pyaudio


In [87]:
!apt-get install -y portaudio19-dev



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
portaudio19-dev is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [88]:

!pip install pyaudio


In [91]:
import speech_recognition as sr

# Initialize recognizer
recognizer = sr.Recognizer()

# Use the microphone as the audio source
with sr.Microphone() as source:
    print("🎤 Please speak something...")
    recognizer.adjust_for_ambient_noise(source)  # Optional: adjust for noise
    audio = recognizer.listen(source)

try:
    # Convert speech to text using Google Web Speech API
    text = recognizer.recognize_google(audio)
    print("📝 You said:", text)
except sr.UnknownValueError:
    print("❌ Could not understand audio.")
except sr.RequestError as e:
    print(f"⚠️ Could not request results from Google; {e}")


OSError: No Default Input Device Available

In [92]:
import tkinter as tk
from tkinter import messagebox

# ---------- Main Application Class ----------
class App:
    def __init__(self, root):
        self.root = root
        self.root.title("User Selection")
        self.root.geometry("400x300")
        self.show_role_selection()

    def show_role_selection(self):
        self.clear_window()
        label = tk.Label(self.root, text="Who are you?", font=("Arial", 16))
        label.pack(pady=20)

        teacher_btn = tk.Button(self.root, text="Teacher", width=20, command=lambda: self.show_form("Teacher"))
        student_btn = tk.Button(self.root, text="Student", width=20, command=lambda: self.show_form("Student"))

        teacher_btn.pack(pady=10)
        student_btn.pack(pady=10)

    def show_form(self, role):
        self.clear_window()
        self.root.title(f"{role} Information Form")

        self.entries = {}
        fields = ["Full Name", "Branch/Class", "School/College", "Phone No.", "Email ID", "Address"]

        tk.Label(self.root, text=f"{role} Information Form", font=("Arial", 14)).pack(pady=10)

        form_frame = tk.Frame(self.root)
        form_frame.pack()

        for field in fields:
            label = tk.Label(form_frame, text=field)
            entry = tk.Entry(form_frame, width=30)
            label.pack()
            entry.pack(pady=3)
            self.entries[field] = entry

        submit_btn = tk.Button(self.root, text="Submit", command=self.submit_form)
        submit_btn.pack(pady=15)

    def submit_form(self):
        data = {field: entry.get() for field, entry in self.entries.items()}

        # Basic validation
        if any(v == "" for v in data.values()):
            messagebox.showerror("Error", "Please fill in all fields.")
            return

        self.show_thank_you()

    def show_thank_you(self):
        self.clear_window()
        tk.Label(self.root, text="✅ Thank you for submitting the form!", font=("Arial", 16), fg="green").pack(pady=60)
        tk.Button(self.root, text="Exit", command=self.root.quit).pack()

    def clear_window(self):
        for widget in self.root.winfo_children():
            widget.destroy()

# ---------- Run the App ----------
if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()


TclError: no display name and no $DISPLAY environment variable

In [93]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.5 MB/s eta 0:00:00


In [94]:
import streamlit as st

st.title("User Information Form")

role = st.radio("Who are you?", ["Teacher", "Student"])

st.subheader(f"{role} Information Form")
name = st.text_input("Full Name")
branch = st.text_input("Branch/Class")
school = st.text_input("School/College")
phone = st.text_input("Phone Number")
email = st.text_input("Email ID")
address = st.text_area("Address")

if st.button("Submit"):
    if not all([name, branch, school, phone, email, address]):
        st.error("Please fill in all the fields.")
    else:
        st.success("✅ Thank you for submitting the form!")


2025-07-12 16:52:28.799 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 16:52:29.321 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-07-12 16:52:29.323 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 16:52:29.327 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 16:52:29.328 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 16:52:29.329 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 16:52:29.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-12 16:52:29.332 Thread 'MainThread': mi

In [96]:
!pip install gradio

import gradio as gr

def submit_form(name, branch, school, phone, email, address):
    return f"✅ Thank you, {name}!\nYour form has been submitted."

with gr.Blocks() as demo:
    gr.Markdown("## Student/Teacher Info Form")

    name = gr.Textbox(label="Full Name")
    branch = gr.Textbox(label="Branch / Class")
    school = gr.Textbox(label="School / College")
    phone = gr.Textbox(label="Phone Number")
    email = gr.Textbox(label="Email ID")
    address = gr.Textbox(label="Address")

    submit = gr.Button("Submit")
    output = gr.Textbox(label="Output")

    submit.click(fn=submit_form,
                 inputs=[name, branch, school, phone, email, address],
                 outputs=output)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://97c2f42521447d84e0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [98]:
!pip install gradio

import gradio as gr

def submit_form(role, full_name, field1, field2, org_name, contact, email, address, extra1, extra2):
    if role == "Student":
        return (f"✅ Thank you, {full_name}!\n\n"
                f"Role: Student\n"
                f"Class: {field1}\n"
                f"School: {org_name}\n"
                f"Principal Name: {extra1}\n"
                f"School Contact: {extra2}\n"
                f"Email: {email}\n"
                f"Phone: {contact}\n"
                f"Address: {address}")
    else:
        return (f"✅ Thank you, {full_name}!\n\n"
                f"Role: Teacher\n"
                f"Branch: {field1}\n"
                f"Subject: {field2}\n"
                f"College: {org_name}\n"
                f"College Location: {extra1}\n"
                f"Degree Certificate: {extra2}\n"
                f"Email: {email}\n"
                f"Phone: {contact}\n"
                f"Address: {address}")

with gr.Blocks() as demo:
    gr.Markdown("## 🎓 Student / 👨‍🏫 Teacher Information Form")

    role = gr.Radio(["Student", "Teacher"], label="Who are you?")

    full_name = gr.Textbox(label="Full Name")

    field1 = gr.Textbox(visible=True)  # Class or Branch
    field2 = gr.Textbox(visible=True)  # (Subject only for teacher)

    org_name = gr.Textbox(label="School/College Name")
    contact = gr.Textbox(label="Phone Number")
    email = gr.Textbox(label="Email ID")
    address = gr.Textbox(label="Address")

    extra1 = gr.Textbox()  # Principal Name or College Location
    extra2 = gr.Textbox()  # School Contact or Degree Certificate

    submit = gr.Button("Submit")
    output = gr.Textbox(label="Output", lines=10)

    def update_fields(role):
        if role == "Student":
            return (
                gr.update(label="Class", visible=True),
                gr.update(label="", visible=False),  # no subject for student
                gr.update(label="Principal Name", visible=True),
                gr.update(label="School Contact No.", visible=True),
                gr.update(label="School Name"),
            )
        else:
            return (
                gr.update(label="Branch", visible=True),
                gr.update(label="Subject", visible=True),
                gr.update(label="College Location", visible=True),
                gr.update(label="Degree Certificate", visible=True),
                gr.update(label="College Name"),
            )

    role.change(fn=update_fields, inputs=role,
                outputs=[field1, field2, extra1, extra2, org_name])

    submit.click(fn=submit_form,
                 inputs=[role, full_name, field1, field2, org_name, contact, email, address, extra1, extra2],
                 outputs=output)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bb340a486df0b35940.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [99]:
!pip install gradio

import gradio as gr

def submit_form(role, full_name, field1, field2, org_name, contact, email, address, extra1, extra2):
    # Check for missing required fields
    required_fields = [role, full_name, field1, org_name, contact, email, address, extra1, extra2]
    if role == "Teacher":
        required_fields.append(field2)  # Subject is required for teachers

    if any(field.strip() == "" for field in required_fields):
        return "❌ Error: Please fill in all required fields."

    # Success message
    if role == "Student":
        return (f"✅ Thank you, {full_name}!\n\n"
                f"Role: Student\n"
                f"Class: {field1}\n"
                f"School: {org_name}\n"
                f"Principal Name: {extra1}\n"
                f"School Contact: {extra2}\n"
                f"Email: {email}\n"
                f"Phone: {contact}\n"
                f"Address: {address}")
    else:
        return (f"✅ Thank you, {full_name}!\n\n"
                f"Role: Teacher\n"
                f"Branch: {field1}\n"
                f"Subject: {field2}\n"
                f"College: {org_name}\n"
                f"College Location: {extra1}\n"
                f"Degree Certificate: {extra2}\n"
                f"Email: {email}\n"
                f"Phone: {contact}\n"
                f"Address: {address}")

with gr.Blocks() as demo:
    gr.Markdown("## 🎓 Student / 👨‍🏫 Teacher Information Form")

    role = gr.Radio(["Student", "Teacher"], label="Who are you?")

    full_name = gr.Textbox(label="Full Name")

    field1 = gr.Textbox(visible=True)  # Class or Branch
    field2 = gr.Textbox(visible=True)  # (Subject only for teacher)

    org_name = gr.Textbox(label="School/College Name")
    contact = gr.Textbox(label="Phone Number")
    email = gr.Textbox(label="Email ID")
    address = gr.Textbox(label="Address")

    extra1 = gr.Textbox()  # Principal Name or College Location
    extra2 = gr.Textbox()  # School Contact or Degree Certificate

    submit = gr.Button("Submit")
    output = gr.Textbox(label="Output", lines=10)

    def update_fields(role):
        if role == "Student":
            return (
                gr.update(label="Class", visible=True),
                gr.update(visible=False),  # Hide subject
                gr.update(label="Principal Name", visible=True),
                gr.update(label="School Contact No.", visible=True),
                gr.update(label="School Name"),
            )
        else:
            return (
                gr.update(label="Branch", visible=True),
                gr.update(label="Subject", visible=True),
                gr.update(label="College Location", visible=True),
                gr.update(label="Degree Certificate", visible=True),
                gr.update(label="College Name"),
            )

    role.change(fn=update_fields, inputs=role,
                outputs=[field1, field2, extra1, extra2, org_name])

    submit.click(fn=submit_form,
                 inputs=[role, full_name, field1, field2, org_name, contact, email, address, extra1, extra2],
                 outputs=output)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7c70e56173957f2401.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [102]:
def update_fields(role):
    if role == "Student":
        return (
            gr.update(label="Class", visible=True),
            gr.update(visible=False),  # hide subject
            gr.update(label="Principal Name", visible=True),
            gr.update(visible=False),  # hide degree upload
            gr.update(label="School Name"),
        )
    else:
        return (
            gr.update(label="Branch", visible=True),
            gr.update(label="Subject", visible=True),
            gr.update(label="College Location", visible=True),
            gr.update(visible=True),  # show degree upload
            gr.update(label="College Name"),
        )


In [105]:
!pip install gradio

import gradio as gr

def submit_form(role, full_name, field1, field2, org_name, contact, email, address, extra1, pdf_file):
    # Check for missing fields
    required_fields = [role, full_name, field1, org_name, contact, email, address, extra1]
    if role == "Teacher":
        required_fields.append(field2)
        if pdf_file is None:
            return "❌ Error: Please upload your degree certificate (PDF)."
    else:
        # Student: fake extra2 as placeholder, not needed anymore
        pdf_file = "School Contact Info Not Uploaded (for student)"

    if any(field.strip() == "" for field in required_fields):
        return "❌ Error: Please fill in all required fields."

    # Success message
    if role == "Student":
        return (f"✅ Thank you, {full_name}!\n\n"
                f"Role: Student\n"
                f"Class: {field1}\n"
                f"School: {org_name}\n"
                f"Principal Name: {extra1}\n"
                f"Email: {email}\n"
                f"Phone: {contact}\n"
                f"Address: {address}")
    else:
        return (f"✅ Thank you, {full_name}!\n\n"
                f"Role: Teacher\n"
                f"Branch: {field1}\n"
                f"Subject: {field2}\n"
                f"College: {org_name}\n"
                f"College Location: {extra1}\n"
                f"Degree Certificate: {pdf_file.name if pdf_file else 'Not Provided'}\n"
                f"Email: {email}\n"
                f"Phone: {contact}\n"
                f"Address: {address}")

with gr.Blocks() as demo:
    gr.Markdown("## 🎓 Student / 👨‍🏫 Teacher Information Form")

    role = gr.Radio(["Student", "Teacher"], label="Who are you?")

    full_name = gr.Textbox(label="Full Name")

    field1 = gr.Textbox(visible=True)  # Class or Branch
    field2 = gr.Textbox(visible=True)  # Subject (teacher only)

    org_name = gr.Textbox(label="School/College Name")
    contact = gr.Textbox(label="Phone Number")
    email = gr.Textbox(label="Email ID")
    address = gr.Textbox(label="Address")

    extra1 = gr.Textbox()  # Principal Name or College Location

    pdf_file = gr.File(label="Upload Degree Certificate (PDF only)", file_types=[".pdf"], visible=False)

    submit = gr.Button("Submit")
    output = gr.Textbox(label="Output", lines=12)

    def update_fields(role):
        if role == "Student":
            return (
                gr.update(label="Class", visible=True),
                gr.update(visible=False),  # hide subject
                gr.update(label="Principal Name", visible=True),
                gr.update(visible=False),  # hide degree upload
                gr.update(label="School Name"),
            )
        else:
            return (
                gr.update(label="Branch", visible=True),
                gr.update(label="Subject", visible=True),
                gr.update(label="College Location", visible=True),
                gr.update(visible=True),  # show degree upload
                gr.update(label="College Name"),
            )



In [97]:
import tkinter as tk
from tkinter import messagebox

# ---------- Main Application Class ----------
class App:
    def __init__(self, root):
        self.root = root
        self.root.title("User Selection")
        self.root.geometry("400x300")
        self.show_role_selection()

    def show_role_selection(self):
        self.clear_window()
        label = tk.Label(self.root, text="Who are you?", font=("Arial", 16))
        label.pack(pady=20)

        teacher_btn = tk.Button(self.root, text="Teacher", width=20, command=lambda: self.show_form("Teacher"))
        student_btn = tk.Button(self.root, text="Student", width=20, command=lambda: self.show_form("Student"))

        teacher_btn.pack(pady=10)
        student_btn.pack(pady=10)

    def show_form(self, role):
        self.clear_window()
        self.root.title(f"{role} Information Form")

        self.entries = {}
        fields = ["Full Name", "Branch/Class", "School/College", "Phone No.", "Email ID", "Address"]

        tk.Label(self.root, text=f"{role} Information Form", font=("Arial", 14)).pack(pady=10)

        form_frame = tk.Frame(self.root)
        form_frame.pack()

        for field in fields:
            label = tk.Label(form_frame, text=field)
            entry = tk.Entry(form_frame, width=30)
            label.pack()
            entry.pack(pady=3)
            self.entries[field] = entry

        submit_btn = tk.Button(self.root, text="Submit", command=self.submit_form)
        submit_btn.pack(pady=15)

    def submit_form(self):
        data = {field: entry.get() for field, entry in self.entries.items()}

        # Basic validation
        if any(v == "" for v in data.values()):
            messagebox.showerror("Error", "Please fill in all fields.")
            return

        self.show_thank_you()

    def show_thank_you(self):
        self.clear_window()
        tk.Label(self.root, text="✅ Thank you for submitting the form!", font=("Arial", 16), fg="green").pack(pady=60)
        tk.Button(self.root, text="Exit", command=self.root.quit).pack()

    def clear_window(self):
        for widget in self.root.winfo_children():
            widget.destroy()

# ---------- Run the App ----------
if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()


TclError: no display name and no $DISPLAY environment variable

In [122]:
!pip install gradio

import gradio as gr

def submit_form(role, full_name, field1, field2, org_name, contact, email, address, extra1, pdf_file):
    # Check for missing fields
    required_fields = [role, full_name, field1, org_name, contact, email, address, extra1]
    if role == "Teacher":
        required_fields.append(field2)
        if pdf_file is None:
            return "❌ Error: Please upload your degree certificate (PDF)."
    else:
        # Student: fake extra2 as placeholder, not needed anymore
        pdf_file = "School Contact Info Not Uploaded (for student)"

    if any(field.strip() == "" for field in required_fields):
        return "❌ Error: Please fill in all required fields."

    # Success message
    if role == "Student":
        return (f"✅ Thank you, {full_name}!\n\n"
                f"Role: Student\n"
                f"Class: {field1}\n"
                f"School: {org_name}\n"
                f"Principal Name: {extra1}\n"
                f"Email: {email}\n"
                f"Phone: {contact}\n"
                f"Address: {address}")
    else:
        return (f"✅ Thank you, {full_name}!\n\n"
                f"Role: Teacher\n"
                f"Branch: {field1}\n"
                f"Subject: {field2}\n"
                f"College: {org_name}\n"
                f"College Location: {extra1}\n"
                f"Degree Certificate: {pdf_file.name if pdf_file else 'Not Provided'}\n"
                f"Email: {email}\n"
                f"Phone: {contact}\n"
                f"Address: {address}")

with gr.Blocks() as demo:
    gr.Markdown("## 🎓 Student / 👨‍🏫 Teacher Information Form")

    role = gr.Radio(["Student", "Teacher"], label="Who are you?")

    full_name = gr.Textbox(label="Full Name")

    field1 = gr.Textbox(visible=True)  # Class or Branch
    field2 = gr.Textbox(visible=True)  # Subject (teacher only)

    org_name = gr.Textbox(label="School/College Name")
    contact = gr.Textbox(label="Phone Number")
    email = gr.Textbox(label="Email ID")
    address = gr.Textbox(label="Address")

    extra1 = gr.Textbox()  # Principal Name or College Location

    pdf_file = gr.File(label="Upload Degree Certificate (PDF only)", file_types=[".pdf"], visible=False)

    submit = gr.Button("Submit")
    output = gr.Textbox(label="Output", lines=12)

    def update_fields(role):
        if role == "Student":
            return (
                gr.update(label="Class", visible=True),
                gr.update(visible=False),  # hide subject
                gr.update(label="Principal Name", visible=True),
                gr.update(visible=False),  # hide degree upload
                gr.update(label="School Name"),
            )
        else:
            return (
                gr.update(label="Branch", visible=True),
                gr.update(label="Subject", visible=True),
                gr.update(label="College Location", visible=True),
                gr.update(label="Degree certificate", visible=True),  # show degree upload
                gr.update(label="College Name"),
            )
        role.change(fn=update_fields, inputs=role,
                outputs=[field1, field2, extra1, extra2, org_name])

        submit.click(fn=submit_form,
                 inputs=[role, full_name, field1, field2, org_name, contact, email, address, extra1, extra2],
                 outputs=output)

    demo.launch()






It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://979ea3fc34f6acf652.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [118]:
!pip install gradio

import gradio as gr

def submit_form(role, full_name, field1, field2, org_name, contact, email, address, extra1, extra2):
    # Check for missing required fields
    required_fields = [role, full_name, field1, org_name, contact, email, address, extra1, extra2]
    if role == "Teacher":
        required_fields.append(field2)
        if pdf_file is none:
          return"❌ Error: Please upload your degree certificate(PDF)."




    if any(field.strip() == "" for field in required_fields):
        return "❌ Error: Please fill in all required fields."

    # Success message
    if role == "Student":
        return (f"✅ Thank you, {full_name}!\n\n"
                f"Role: Student\n"
                f"Class: {field1}\n"
                f"School: {org_name}\n"
                f"Principal Name: {extra1}\n"
                f"School Contact: {extra2}\n"
                f"Email: {email}\n"
                f"Phone: {contact}\n"
                f"Address: {address}")
    else:
        return (f"✅ Thank you, {full_name}!\n\n"
                f"Role: Teacher\n"
                f"Branch: {field1}\n"
                f"Subject: {field2}\n"
                f"College: {org_name}\n"
                f"College Location: {extra1}\n"
                f"Degree Certificate: {pdf_file.name if pdf_file else 'Not Provided'}\n"
                f"Email: {email}\n"
                f"Phone: {contact}\n"
                f"Address: {address}")

with gr.Blocks() as demo:
    gr.Markdown("## 🎓 Student / 👨‍🏫 Teacher Information Form")

    role = gr.Radio(["Student", "Teacher"], label="Who are you?")

    full_name = gr.Textbox(label="Full Name")

    field1 = gr.Textbox(visible=True)  # Class or Branch
    field2 = gr.Textbox(visible=True)  # (Subject only for teacher)

    org_name = gr.Textbox(label="School/College Name")
    contact = gr.Textbox(label="Phone Number")
    email = gr.Textbox(label="Email ID")
    address = gr.Textbox(label="Address")

    extra1 = gr.Textbox()  # Principal Name or College Location

   pdf_file = gr.File(label="Upload Degree Certificate (PDF only)", file_types=[".pdf"], visible=false)

    submit = gr.Button("Submit")
    output = gr.Textbox(label="Output", lines=12)

    def update_fields(role):
        if role == "Student":
            return (
                gr.update(label="Class", visible=True),
                gr.update(visible=False),  # Hide subject
                gr.update(label="Principal Name", visible=True),
                gr.update(label="School Contact No.", visible=True),
                gr.update(label="School Name"),
            )
        else:
            return (
                gr.update(label="Branch", visible=True),
                gr.update(label="Subject", visible=True),
                gr.update(label="College Location", visible=True),
                gr.update(visible=True),
                gr.update(label="College Name"),
            )

    role.change(fn=update_fields, inputs=role,
                outputs=[field1, field2, extra1, extra2, org_name])

    submit.click(fn=submit_form,
                 inputs=[role, full_name, field1, field2, org_name, contact, email, address, extra1, extra2],
                 outputs=output)

demo.launch()


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 59)

In [124]:
!pip install gradio

import gradio as gr

def submit_form(role, full_name, field1, field2, org_name, contact, email, address, extra1, extra2, pdf_file):
    # Check for missing required fields
    required_fields = [role, full_name, field1, org_name, contact, email, address, extra1, extra2]

    if role == "Teacher":
        required_fields.append(field2)
        if pdf_file is None:
            return "❌ Error: Please upload your degree certificate (PDF)."

    if any(field.strip() == "" for field in required_fields if isinstance(field, str)):
        return "❌ Error: Please fill in all required fields."

    # Success message
    if role == "Student":
        return (f"✅ Thank you, {full_name}!\n\n"
                f"Role: Student\n"
                f"Class: {field1}\n"
                f"School: {org_name}\n"
                f"Principal Name: {extra1}\n"
                f"School Contact: {extra2}\n"
                f"Email: {email}\n"
                f"Phone: {contact}\n"
                f"Address: {address}")
    else:
        return (f"✅ Thank you, {full_name}!\n\n"
                f"Role: Teacher\n"
                f"Branch: {field1}\n"
                f"Subject: {field2}\n"
                f"College: {org_name}\n"
                f"College Location: {extra1}\n"
                f"Degree Certificate: {pdf_file.name if pdf_file else 'Not Provided'}\n"
                f"Email: {email}\n"
                f"Phone: {contact}\n"
                f"Address: {address}")

with gr.Blocks() as demo:
    gr.Markdown("## 🎓 Student / 👨‍🏫 Teacher Information Form")

    role = gr.Radio(["Student", "Teacher"], label="Who are you?")

    full_name = gr.Textbox(label="Full Name")

    field1 = gr.Textbox(visible=True)  # Class or Branch
    field2 = gr.Textbox(visible=True)  # Subject (teacher only)

    org_name = gr.Textbox(label="School/College Name")
    contact = gr.Textbox(label="Phone Number")
    email = gr.Textbox(label="Email ID")
    address = gr.Textbox(label="Address")

    extra1 = gr.Textbox()  # Principal Name or College Location
    extra2 = gr.Textbox()  # School Contact No. or Degree Certificate text if any

    pdf_file = gr.File(label="Upload Degree Certificate (PDF only)", file_types=[".pdf"], visible=False)

    submit = gr.Button("Submit")
    output = gr.Textbox(label="Output", lines=12)

    def update_fields(role):
        if role == "Student":
            return (
                gr.update(label="Class", visible=True),
                gr.update(visible=False),  # Hide subject
                gr.update(label="Principal Name", visible=True),
                gr.update(label="School Contact No.", visible=True),
                gr.update(visible=False),  # Hide PDF upload
                gr.update(label="School Name")
            )
        else:
            return (
                gr.update(label="Branch", visible=True),
                gr.update(label="Subject", visible=True),
                gr.update(label="College Location", visible=True),
                gr.update(label="Degree Certificate", visible=True),
                gr.update(visible=True),
                gr.update(label="College Name")
            )

    role.change(fn=update_fields, inputs=role,
                outputs=[field1, field2, extra1, extra2, pdf_file, org_name])

    submit.click(fn=submit_form,
                 inputs=[role, full_name, field1, field2, org_name, contact, email, address, extra1, extra2, pdf_file],
                 outputs=output)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9d5c8ee72695b3f4c8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [125]:
!pip install gradio

import gradio as gr

def submit_form(role, full_name, field1, field2, org_name, contact, email, address, extra1, extra2, pdf_file):
    # Check for missing required fields
    required_fields = [role, full_name, field1, org_name, contact, email, address, extra1, extra2]

    if role == "Teacher":
        required_fields.append(field2)
        if pdf_file is None:
            return "❌ Error: Please upload your degree certificate (PDF)."

    if any(field.strip() == "" for field in required_fields if isinstance(field, str)):
        return "❌ Error: Please fill in all required fields."

    # Success message
    if role == "Student":
        return (f"✅ Thank you, {full_name}!\n\n"
                f"Role: Student\n"
                f"Class: {field1}\n"
                f"School: {org_name}\n"
                f"Principal Name: {extra1}\n"
                f"School Contact: {extra2}\n"
                f"Email: {email}\n"
                f"Phone: {contact}\n"
                f"Address: {address}\n\n"
                f"🔍 Now enter a topic below to search.")
    else:
        return (f"✅ Thank you, {full_name}!\n\n"
                f"Role: Teacher\n"
                f"Branch: {field1}\n"
                f"Subject: {field2}\n"
                f"College: {org_name}\n"
                f"College Location: {extra1}\n"
                f"Degree Certificate: {pdf_file.name if pdf_file else 'Not Provided'}\n"
                f"Email: {email}\n"
                f"Phone: {contact}\n"
                f"Address: {address}\n\n"
                f"🔍 Now enter a topic below to search.")

def search_topic_fn(topic):
    if topic.strip() == "":
        return "❌ Please enter a topic to search."
    return f"🔎 You want to search: **{topic}**"

with gr.Blocks() as demo:
    gr.Markdown("## 🎓 Student / 👨‍🏫 Teacher Information Form")

    role = gr.Radio(["Student", "Teacher"], label="Who are you?")

    full_name = gr.Textbox(label="Full Name")

    field1 = gr.Textbox(visible=True)  # Class or Branch
    field2 = gr.Textbox(visible=True)  # Subject (teacher only)

    org_name = gr.Textbox(label="School/College Name")
    contact = gr.Textbox(label="Phone Number")
    email = gr.Textbox(label="Email ID")
    address = gr.Textbox(label="Address")

    extra1 = gr.Textbox()  # Principal Name or College Location
    extra2 = gr.Textbox()  # School Contact No. or placeholder

    pdf_file = gr.File(label="Upload Degree Certificate (PDF only)", file_types=[".pdf"], visible=False)

    submit = gr.Button("Submit")
    output = gr.Textbox(label="Form Response", lines=12)

    # NEW: Search topic area
    gr.Markdown("## 🔍 Enter Topic to Search")
    search_topic = gr.Textbox(label="Search Topic")
    search_button = gr.Button("Search")
    search_output = gr.Textbox(label="Search Output", lines=2)

    def update_fields(role):
        if role == "Student":
            return (
                gr.update(label="Class", visible=True),
                gr.update(visible=False),  # Hide subject
                gr.update(label="Principal Name", visible=True),
                gr.update(label="School Contact No.", visible=True),
                gr.update(visible=False),  # Hide PDF upload
                gr.update(label="School Name")
            )
        else:
            return (
                gr.update(label="Branch", visible=True),
                gr.update(label="Subject", visible=True),
                gr.update(label="College Location", visible=True),
                gr.update(label="Degree Certificate", visible=True),
                gr.update(visible=True),
                gr.update(label="College Name")
            )

    role.change(fn=update_fields, inputs=role,
                outputs=[field1, field2, extra1, extra2, pdf_file, org_name])

    submit.click(fn=submit_form,
                 inputs=[role, full_name, field1, field2, org_name, contact, email, address, extra1, extra2, pdf_file],
                 outputs=output)

    search_button.click(fn=search_topic_fn, inputs=search_topic, outputs=search_output)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://db6eb68d165739f3a0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [126]:
!pip install gradio

import gradio as gr
import urllib.parse

def submit_form(role, full_name, field1, field2, org_name, contact, email, address, extra1, extra2, pdf_file):
    required_fields = [role, full_name, field1, org_name, contact, email, address, extra1, extra2]

    if role == "Teacher":
        required_fields.append(field2)
        if pdf_file is None:
            return "❌ Error: Please upload your degree certificate (PDF)."

    if any(field.strip() == "" for field in required_fields if isinstance(field, str)):
        return "❌ Error: Please fill in all required fields."

    if role == "Student":
        return (f"✅ Thank you, {full_name}!\n\n"
                f"Role: Student\n"
                f"Class: {field1}\n"
                f"School: {org_name}\n"
                f"Principal Name: {extra1}\n"
                f"School Contact: {extra2}\n"
                f"Email: {email}\n"
                f"Phone: {contact}\n"
                f"Address: {address}\n\n"
                f"🔍 Now enter a topic below to search.")
    else:
        return (f"✅ Thank you, {full_name}!\n\n"
                f"Role: Teacher\n"
                f"Branch: {field1}\n"
                f"Subject: {field2}\n"
                f"College: {org_name}\n"
                f"College Location: {extra1}\n"
                f"Degree Certificate: {pdf_file.name if pdf_file else 'Not Provided'}\n"
                f"Email: {email}\n"
                f"Phone: {contact}\n"
                f"Address: {address}\n\n"
                f"🔍 Now enter a topic below to search.")

def search_topic_fn(topic):
    if topic.strip() == "":
        return "❌ Please enter a topic to search.", ""

    # Encode topic for URLs
    encoded = urllib.parse.quote(topic)

    # Links
    yt_link = f"https://www.youtube.com/results?search_query={encoded}"
    google_notes_link = f"https://www.google.com/search?q=handwritten+notes+on+{encoded}"

    # Simulated ChatGPT answer
    simulated_answer = f"""📘 **AI Summary for:** {topic}

✅ This topic generally includes:
- Introduction and key definitions.
- Core concepts and examples.
- Applications or use cases.

👉 Visit YouTube for video explanations.
👉 Use Google to find handwritten notes or PDFs.
"""

    links = f"""
🔗 **Resources for "{topic}":**

- 📺 [YouTube Videos]({yt_link})
- 📖 [Google Handwritten Notes]({google_notes_link})
"""

    return simulated_answer, links

with gr.Blocks() as demo:
    gr.Markdown("## 🎓 Student / 👨‍🏫 Teacher Information Form")

    role = gr.Radio(["Student", "Teacher"], label="Who are you?")
    full_name = gr.Textbox(label="Full Name")
    field1 = gr.Textbox(visible=True)  # Class or Branch
    field2 = gr.Textbox(visible=True)  # Subject (teacher only)

    org_name = gr.Textbox(label="School/College Name")
    contact = gr.Textbox(label="Phone Number")
    email = gr.Textbox(label="Email ID")
    address = gr.Textbox(label="Address")
    extra1 = gr.Textbox()  # Principal Name or College Location
    extra2 = gr.Textbox()  # School Contact or placeholder
    pdf_file = gr.File(label="Upload Degree Certificate (PDF only)", file_types=[".pdf"], visible=False)

    submit = gr.Button("Submit")
    output = gr.Textbox(label="Form Response", lines=12)

    gr.Markdown("## 🔍 Enter Topic to Search Resources")
    search_topic = gr.Textbox(label="Search Topic")
    search_button = gr.Button("Search")
    search_answer = gr.Textbox(label="AI Summary", lines=5)
    search_links = gr.Markdown()

    def update_fields(role):
        if role == "Student":
            return (
                gr.update(label="Class", visible=True),
                gr.update(visible=False),
                gr.update(label="Principal Name", visible=True),
                gr.update(label="School Contact No.", visible=True),
                gr.update(visible=False),
                gr.update(label="School Name")
            )
        else:
            return (
                gr.update(label="Branch", visible=True),
                gr.update(label="Subject", visible=True),
                gr.update(label="College Location", visible=True),
                gr.update(label="Degree Certificate", visible=True),
                gr.update(visible=True),
                gr.update(label="College Name")
            )

    role.change(fn=update_fields, inputs=role,
                outputs=[field1, field2, extra1, extra2, pdf_file, org_name])

    submit.click(fn=submit_form,
                 inputs=[role, full_name, field1, field2, org_name, contact, email, address, extra1, extra2, pdf_file],
                 outputs=output)

    search_button.click(fn=search_topic_fn, inputs=search_topic,
                        outputs=[search_answer, search_links])

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d9f344b4489f03fcd4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
